In [1]:
import riskfolio as rp 
import pandas as pd
import numpy as np
from openbb_terminal.sdk import openbb
import datetime
from dateutil.relativedelta import relativedelta
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import yfinance as yf
import bt 
import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(2)

data = sheet_instance.get_all_values()

portfolio = pd.DataFrame(data)

headers = portfolio.iloc[0]
portfolio = pd.DataFrame(portfolio.values[1:], columns=headers)

portfolio = portfolio.rename(columns= {'':'Ticker'})

portfolio['Shares'] = portfolio['Shares'].astype(float)

total_shares = portfolio['Shares'].sum()

portfolio['% Allocation'] = (portfolio['Shares'] / total_shares)

portfolio



,Ticker,Shares,% Allocation
0,CEG,2108.0,0.108576
1,AAPL,1727.0,0.088952
2,ULTA,1685.0,0.086789
3,MSFT,1657.0,0.085346
4,RTX,1623.0,0.083595
5,JPM,1321.0,0.068040
6,CVX,1276.0,0.065722
7,PANW,1225.0,0.063096
8,GEHC,1141.0,0.058769
9,AMZN,1080.0,0.055627


In [3]:
end = pd.Timestamp(datetime.date.today())
start = end - relativedelta(years=1)

symbols = portfolio['Ticker'].tolist()

tickers = openbb.stocks.ca.hist(symbols,start, end)

returns = tickers.pct_change()[1:]
returns.dropna(how="any", axis=1, inplace=True)

NaN values found in: NVDA.  Backfilling data

In [22]:
risk_measures = ['MV', 'SLPM', 'CVaR','MAD','FLPM','EVaR','UCI','CDaR','MSV']

weights = pd.DataFrame([])

# Create an instance of the Portfolio class
P = rp.Portfolio(returns=returns,)

# Define constraints
P.assets_stats(method_mu='hist', method_cov='hist', d=0.94)

P.lowerret = 0.0020

for rm in risk_measures:

    w_rp = P.rp_optimization(model='Classic', rm = rm, b=None)
    
    weights = pd.concat([weights, w_rp], axis=1)

weights.columns = risk_measures

weights_deindex = weights.reset_index()

weights_deindex.rename(columns={'index':'Ticker'}, inplace=True )

weights_deindex

,Ticker,MV,SLPM,CVaR,MAD,FLPM,EVaR,UCI,CDaR,MSV
0,CEG,0.079997,0.081071,0.070374,0.079150,0.083618,0.075354,0.205135,0.204318,0.078631
1,AAPL,0.059313,0.061414,0.074040,0.059896,0.059691,0.073395,0.042028,0.046701,0.060901
2,ULTA,0.045212,0.048562,0.050929,0.044139,0.044458,0.057466,0.038021,0.038366,0.048000
3,MSFT,0.080585,0.083279,0.100133,0.080522,0.082357,0.083813,0.057459,0.075028,0.081124
4,RTX,0.044430,0.048566,0.045544,0.047054,0.046517,0.042612,0.028242,0.030987,0.048128
5,JPM,0.066042,0.068993,0.059129,0.060411,0.062980,0.068484,0.066614,0.060232,0.066436
6,CVX,0.038769,0.041697,0.040044,0.041383,0.039160,0.038291,0.049486,0.040880,0.041827
7,PANW,0.117359,0.092814,0.089650,0.113527,0.115632,0.065790,0.111629,0.113815,0.095369
8,GEHC,0.041589,0.045445,0.047305,0.042070,0.042646,0.042031,0.041275,0.037357,0.045518
9,AMZN,0.067049,0.054334,0.051605,0.061945,0.058036,0.062525,0.059488,0.048019,0.056542


In [5]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(1)

#mapping_list =weights_deindex.values.tolist()

#sheet.values_append('RM Weights!A1',{'valueInputOption' : 'RAW'}, {'values':mapping_list})

set_with_dataframe(sheet_instance,weights_deindex)

In [6]:
stock_data = yf.download(symbols, start= start, end= end)['Adj Close']

stock_data.dropna(how="any", axis=1, inplace=True)

port_allocation = portfolio['% Allocation'].tolist()

port_list = portfolio['% Allocation'].tolist()

port_values = portfolio['% Allocation'].values

# Fetch the S&P 500 index data
sp500_data = yf.download('^GSPC', start= start, end= end)['Adj Close']

sp500_data = sp500_data.to_frame(name='S&P500')


[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
results = {}

# Assuming stock_data is a DataFrame with historical price data for each asset
# stock_data.columns should match the assets in the weights DataFrame

for rm in risk_measures:
    # Extracting the allocation for the given risk measure
    allocation = weights[rm].values

    # Ensuring the weights sum up to 1 (if they don't due to rounding or other reasons)
    allocation = allocation/allocation.sum()

    # Create the portfolio strategy for the given risk measure
    portfolio_weights = dict(zip(stock_data, allocation))
    
    portfolio_strategy = bt.Strategy( f'Portfolio_{rm}',
                                     algos=[
                                         bt.algos.SelectAll(),
                                         bt.algos.WeighSpecified(**portfolio_weights),
                                         bt.algos.Rebalance()
                                     ])

    # Backtest the portfolio strategy
    portfolio_backtest = bt.Backtest(portfolio_strategy, stock_data)
    res = bt.run(portfolio_backtest)

    # Store the backtest result for the risk measure in the results dictionary
    results[rm] = res

 
stats_dict = {}

for rm, res in results.items():
    # Extracting stats for each backtest. This returns a pandas Series
    stats_series = res.stats
    
    # Store the Series in the dictionary with risk measure as the key
    stats_dict[rm] = stats_series

stats_df = pd.concat(stats_dict, axis=1).T

# Transpose again to swap the index with the columns
stats_df = stats_df.T

stats_df.columns = stats_df.columns.get_level_values(0) 

RM_results = stats_df 


   

In [8]:
# Create the portfolio strategy
portfolio_weights = dict(zip(stock_data, port_values))
portfolio_strategy = bt.Strategy('MyPortfolio', 
                                  algos=[
                                      bt.algos.SelectAll(),
                                      bt.algos.WeighSpecified(**portfolio_weights),
                                      bt.algos.Rebalance()
                                  ])

# Backtest the portfolio strategy
portfolio_backtest = bt.Backtest(portfolio_strategy, stock_data)


In [9]:
# Create and backtest the S&P 500 benchmark
benchmark_strategy = bt.Strategy('S&P500', [bt.algos.RunOnce(),
                                            bt.algos.SelectAll(),
                                            bt.algos.WeighEqually(),
                                            bt.algos.Rebalance()])
benchmark_backtest = bt.Backtest(benchmark_strategy, sp500_data)

In [10]:
# Create and backtest the Equal Allocation benchmark
equal_benchmark_strategy = bt.Strategy('EqualAllocation', [bt.algos.RunOnce(),
                                            bt.algos.SelectAll(),
                                            bt.algos.WeighEqually(),
                                            bt.algos.Rebalance()])
equal_benchmark_backtest = bt.Backtest(equal_benchmark_strategy, stock_data)

In [11]:
# Run the backtests and compare the results
res = bt.run(portfolio_backtest, equal_benchmark_backtest, benchmark_backtest)

# Example: Get the equity curve for the first backtest strategy
equity = res[0].prices

# Calculate the daily percentage returns
daily_returns = equity.pct_change() * 100

stats = res.stats

df_stats = pd.DataFrame(stats)

BM_results = df_stats


In [12]:
portfolio_df = pd.concat([RM_results, BM_results], axis = 1)

portfolio_df = portfolio_df.iloc[:-33]

backtested_portfolio = portfolio_df.iloc[3:]

backtested_portfolio = backtested_portfolio.reset_index()

backtested_portfolio.rename(columns={'index':'Results'}, inplace=True )

backtested_portfolio



,Results,MV,SLPM,CVaR,MAD,FLPM,EVaR,UCI,CDaR,MSV,MyPortfolio,EqualAllocation,S&P500
0,total_return,0.344936,0.358253,0.378818,0.34717,0.347638,0.399845,0.370121,0.34868,0.352211,0.35499,0.416942,0.205098
1,cagr,0.345209,0.358538,0.379121,0.347445,0.347913,0.400167,0.370416,0.348956,0.35249,0.355272,0.417281,0.205252
2,max_drawdown,-0.115279,-0.112453,-0.10778,-0.117933,-0.114856,-0.101725,-0.122271,-0.123913,-0.113239,-0.105207,-0.119324,-0.102663
3,calmar,2.994542,3.188332,3.517538,2.946124,3.029129,3.933804,3.029473,2.816144,3.112809,3.376898,3.49704,1.999275
4,mtd,0.042587,0.044135,0.046039,0.043564,0.043356,0.048837,0.037025,0.03295,0.042694,0.042204,0.068996,0.026041
5,three_month,0.195676,0.196795,0.202785,0.199987,0.198995,0.200037,0.198218,0.189778,0.195572,0.194539,0.239281,0.168773
6,six_month,0.080949,0.08465,0.094835,0.082217,0.084081,0.1008,0.075606,0.065519,0.082737,0.089734,0.121831,0.071456
7,ytd,0.042587,0.044135,0.046039,0.043564,0.043356,0.048837,0.037025,0.03295,0.042694,0.042204,0.068996,0.026041
8,one_year,0.344936,0.358253,0.378818,0.34717,0.347638,0.399845,0.370121,0.34868,0.352211,0.35499,0.416942,0.205098
9,three_year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(3)

# set dataframe for initial write the use the appending code below for next iterations 
set_with_dataframe(sheet_instance,backtested_portfolio)

In [18]:

merged_df = portfolio.merge(weights_deindex[['Ticker', 'CVaR']], on='Ticker', how='left')

#sum of total portfolio value
sum = merged_df['Shares'].sum()

weekly_invest = 150

merged_df['CVaR Allocation'] = merged_df['CVaR'] * sum

merged_df['$ Diff'] = merged_df['CVaR Allocation'] - merged_df['Shares']

merged_df['Weekly_Contribution'] = merged_df['CVaR'] * weekly_invest

merged_df['date'] = pd.Timestamp.now().date()

merged_df = merged_df.sort_values(by='$ Diff', ascending = True)

merged_df



,Ticker,Shares,% Allocation,CVaR,CVaR Allocation,$ Diff,Weekly_Contribution,date
0,CEG,2108.0,0.108576,0.070921,1376.925306,-731.074694,10.638104,2024-01-26
7,PANW,1225.0,0.063096,0.039061,758.375903,-466.624097,5.859201,2024-01-26
1,AAPL,1727.0,0.088952,0.080043,1554.042536,-172.957464,12.006509,2024-01-26
13,NOW,857.0,0.044141,0.036760,713.687062,-143.312938,5.513936,2024-01-26
12,NVDA,1004.0,0.051713,0.046217,897.310174,-106.689826,6.932605,2024-01-26
3,MSFT,1657.0,0.085346,0.081443,1581.213576,-75.786424,12.216432,2024-01-26
9,AMZN,1080.0,0.055627,0.053640,1041.427960,-38.572040,8.046057,2024-01-26
4,RTX,1623.0,0.083595,0.084394,1638.514241,15.514241,12.659137,2024-01-26
5,JPM,1321.0,0.068040,0.072817,1413.750282,92.750282,10.922614,2024-01-26
2,ULTA,1685.0,0.086789,0.095052,1845.433037,160.433037,14.257788,2024-01-26


In [20]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(0)

existing_data = sheet_instance.get_all_records()
start_row = len(existing_data) + 2

# set dataframe for initial write the use the appending code below for next iterations 
set_with_dataframe(sheet_instance,merged_df,row=start_row,include_column_header=False)



#mapping_list =merged_df.values.tolist()

#sheet.values_append('Allocation Records!A1',{'valueInputOption' : 'RAW'}, {'values':mapping_list})



